In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

%matplotlib inline
%run function.ipynb

df = pd.read_csv('clean8.csv')
df_copy = df.copy()
print(df_copy.shape)
df_copy.head(3)

(855794, 54)


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,USDPerJobCreated,loan_marturity_date,Is_Cross_State,Appv_Dusburse_delta,Default%,RealEstate,GreatRecession,DotCom,Recession,Mon_Is_14710
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,1997-02-28,84.0,4.0,...,NaN,2004-02-28,1,730.0,NaN,0.0,0.0,0.0,0.0,0.0
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,1997-02-28,60.0,2.0,...,NaN,2002-02-28,0,92.0,NaN,0.0,0.0,0.0,0.0,0.0
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,1997-02-28,180.0,7.0,...,NaN,2012-02-28,0,306.0,NaN,0.0,0.0,0.0,0.0,0.0


In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df_copy = df_copy[(df_copy['DisbursementDate_year'] >= 2000) & (df_copy['DisbursementDate_year'] <= 2010)]
print(df_copy.shape)
df_copy.head(3)

(530453, 54)


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,USDPerJobCreated,loan_marturity_date,Is_Cross_State,Appv_Dusburse_delta,Default%,RealEstate,GreatRecession,DotCom,Recession,Mon_Is_14710
12,1000146010,CARVEL,APEX,NC,27502,STEARNS BK NATL ASSOC,MN,2006-02-07,162.000,2.000,...,NaN,2019-08-07,1,52.000,NaN,0.000,0.000,0.000,0.000,0.000
24,1000446006,SUBWAY,LITTLE ROCK,AR,72223,HOPE FCU,MS,2006-02-07,126.000,7.000,...,NaN,2016-08-07,1,82.000,NaN,0.000,0.000,0.000,0.000,1.000
28,1000506000,WEYLAND CORPORATION,CAMARILLO,CA,93010,WELLS FARGO BANK NATL ASSOC,SD,2006-02-07,83.000,18.000,...,106321.347,2013-01-07,1,21.000,NaN,0.000,0.000,0.000,0.000,0.000


In [4]:
df_copy['DisbursePerMonth'] = df_copy['DisbursementGross'] / df_copy['Term']
df_copy['BankLoss'] = df_copy['ChgOffPrinGr'] - df_copy['SBA_Appv']
df_copy.loc[list(df_copy[df_copy['BankLoss'] < 0].index), 'BankLoss'] = 0
df_copy['BankLoss%'] = df_copy['BankLoss'] / df_copy['DisbursementGross'] * 100
df_copy['Job'] = df_copy['CreateJob'] + df_copy['RetainedJob']

In [5]:
df_copy['SBALoss'] = df_copy['ChgOffPrinGr'] - df_copy['BankLoss']

In [6]:
obj_features = ['State', 'BankState', 'NewExist', 'Bank', 'IsFranchised', 'UrbanRural', 'RevLineCr', 'LowDoc', 'Industry',
                'ApprovalDate_year', 'ApprovalDate_month', 'DisbursementDate_year', 'DisbursementDate_month', 'State_division', 'RealEstate',
                'GreatRecession', 'DotCom', 'Recession', 'Mon_Is_14710', 'Is_Cross_State']
num_features = ['Default', 'ChgOffPrinGr', 'Term', 'DisbursementGross', 'Bank%', 'DisbursePerEmployee', 'BankLoss', 'BankLoss%', 'Job',
                'USDPerJobCreated','NoEmp', 'CreateJob', 'RetainedJob', 'Population', 'PopDensity', 'GrAppv', 'SBA_Appv', 'Default%',
                'HousingUnits', 'OccupiedHousingUnits', 'MedianHomeValue', 'MedianHouseholdIncome', 'Appv_Dusburse_delta', 'DisbursePerMonth', 'SBALoss']
time_cols = ['ApprovalDate_year', 'ApprovalDate_month', 'DisbursementDate_year', 'DisbursementDate_month']

df_groupby_mean = dict()
df_groupby_size = dict()
df_groupby_sum = dict()

for col in obj_features:
    df_groupby_mean[col] = df_copy.groupby(by=col).mean()[num_features]
    df_groupby_sum[col] = df_copy.groupby(by=col).sum()[num_features]
    df_groupby_size[col] = pd.DataFrame(df_copy.groupby(by=col).size())
    df_groupby_size[col].rename(columns={0: 'size'}, inplace=True)

In [7]:
display(df_copy.iloc[:3,:23])
display(df_copy.iloc[:3,23:])

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,IsFranchised,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,Default,ChgOffPrinGr
12,1000146010,CARVEL,APEX,NC,27502,STEARNS BK NATL ASSOC,MN,2006-02-07,162.000,2.000,...,1.000,1,0.000,0.000,NaN,2006-03-31,307175.719,0.000,0.000,NaN
24,1000446006,SUBWAY,LITTLE ROCK,AR,72223,HOPE FCU,MS,2006-02-07,126.000,7.000,...,0.000,1,0.000,0.000,NaN,2006-04-30,166437.357,0.000,0.000,NaN
28,1000506000,WEYLAND CORPORATION,CAMARILLO,CA,93010,WELLS FARGO BANK NATL ASSOC,SD,2006-02-07,83.000,18.000,...,0.000,1,1.000,0.000,NaN,2006-02-28,531606.737,0.000,0.000,NaN


,GrAppv,SBA_Appv,Industry,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,...,RealEstate,GreatRecession,DotCom,Recession,Mon_Is_14710,DisbursePerMonth,BankLoss,BankLoss%,Job,SBALoss
12,307175.719,230381.789,Retail_trade,NaN,NaN,NaN,2006.000,3.000,31.000,2006.000,...,0.000,0.000,0.000,0.000,0.000,1896.146,NaN,NaN,0.000,NaN
24,166437.357,141471.753,Accom/Food_serv,NaN,NaN,NaN,2006.000,4.000,30.000,2006.000,...,0.000,0.000,0.000,0.000,1.000,1320.931,NaN,NaN,0.000,NaN
28,121221.673,60610.836,Educational,NaN,NaN,NaN,2006.000,2.000,28.000,2006.000,...,0.000,0.000,0.000,0.000,0.000,6404.900,NaN,NaN,28.000,NaN


In [8]:
# num = 'Default'
# for i in range(len(obj_features)):
#     if df_copy[obj_features[i]].nunique() < 8:
#             plt.figure(figsize=(10, 10))
#     elif obj_features[i] == 'Industry':
#         plt.figure(figsize=(20, 15))
#         vis = df_groupby_mean[obj_features[i]].sort_values(by=num, ascending=False)
#         title = 'Mean Default by '+ obj_features[i]
#         plt.title(title)
#         sns.set_theme(style="darkgrid")
#         plt.grid(True)
#         sns.barplot(y=vis.index, x=vis[num])
#         plt.savefig(title + '.jpeg')
#         plt.show()
#         continue
#     else:
#         plt.figure(figsize=(20, 10))
#     vis = df_groupby_mean[obj_features[i]].sort_values(by=num, ascending=False)
#     title = 'Mean Default by '+ obj_features[i]
#     plt.title(title)
#     sns.set_theme(style="darkgrid")
#     plt.grid(True)
#     sns.barplot(x=vis.index, y=vis[num])
#     plt.savefig(title + '.jpeg')
#     plt.show()

In [9]:
# num_visualization_with_hue(df_copy, num_features, hue_col='Default', exclude_outlier=2.5)

In [10]:
# df_vis = df_copy.copy()
# hue_col = 'Default'

# for i in range(len(num_features)):
#     if num_features[i] in [hue_col, 'ChgOffPrinGr']:
#         continue

#     num_col = num_features[i]
#     vis = df_vis[[num_col, hue_col]]
#     vis = vis[~(pd.isnull(vis[num_features[i]]))]

#     upb = np.percentile(vis, 97.5)
#     lwb = np.percentile(vis, 2.5)
#     vis = vis[(vis[num_col] >= lwb) & (vis[num_col] <= upb)]

#     plt.figure(figsize=(20, 10))
#     sns.set_theme(style="darkgrid")
#     title = num_col + '_ecdf'
#     plt.title(title)
#     plt.hlines(y=0.5, xmin=vis[num_col].min(), xmax=vis[num_col].max(), colors='r', linestyles='dashed')
#     sns.set_palette(sns.color_palette("Set2"))
#     sns.ecdfplot(data=vis, x=num_col, hue='Default')
#     plt.savefig(title + '.jpeg')
#     plt.show()
#     df_vis = df_copy.copy()

In [11]:
# df_vis = df_copy.copy()
# hue_col = 'Default'
# db = ['DisbursementGross']

# for i in range(len(db)):
#     if db[i] in [hue_col, 'ChgOffPrinGr']:
#         continue

#     num_col = db[i]
#     vis = df_vis[[num_col, hue_col]]
#     vis = vis[~(pd.isnull(vis[db[i]]))]
    
#     upb = np.percentile(vis, 95)
#     lwb = np.percentile(vis, 5)
#     vis = vis[(vis[num_col] >= lwb) & (vis[num_col] <= upb)]
    
#     plt.figure(figsize=(20, 10))
#     sns.set_theme(style="darkgrid")
#     title = num_col + '_ecdf'
#     plt.title(title)
#     plt.hlines(y=0.5, xmin=vis[num_col].min(), xmax=vis[num_col].max(), colors='r', linestyles='dashed')
#     sns.ecdfplot(data=vis, x=num_col, hue='Default', palette=sns.color_palette("Set2")[:2])
#     plt.savefig(title + '.jpeg')
#     plt.show()
#     df_vis = df_copy.copy()

In [12]:
# num_visualization(df_copy, num_features=['BankLoss'], exclude_outlier=2.5)

In [13]:
# num_col = 'BankLoss'
# df_vis = df_copy.copy()
# df_vis = df_vis[~(pd.isnull(df_vis[num_col]))]

# upb = np.percentile(df_vis[num_col], 99.5)
# df_vis = df_vis[(df_vis[num_col] <= upb)]

# plt.figure(figsize=(20, 10))
# sns.set_theme(style="darkgrid")
# title = num_col + '_ecdf'
# plt.title(title)
# plt.hlines(y=0.5, xmin=df_vis[num_col].min(), xmax=df_vis[num_col].max(), colors='r', linestyles='dashed')
# sns.ecdfplot(data=df_vis, x=num_col, palette=sns.color_palette("Set2")[:2])
# plt.savefig(title + '.jpeg')
# plt.show()

In [14]:
# num_col = 'BankLoss%'
# df_vis = df_copy.copy()
# df_vis = df_vis[~(pd.isnull(df_vis[num_col]))]

# # upb = np.percentile(df_vis[num_col], 99)
# # df_vis = df_vis[(df_vis[num_col] <= upb)]

# plt.figure(figsize=(20, 10))
# sns.set_theme(style="darkgrid")
# title = num_col + '_ecdf'
# plt.title(title)
# plt.hlines(y=0.5, xmin=df_vis[num_col].min(), xmax=df_vis[num_col].max(), colors='r', linestyles='dashed')
# sns.ecdfplot(data=df_vis, x=num_col, palette=sns.color_palette("Set2")[:2])
# plt.savefig(title + '.jpeg')
# plt.show()

In [15]:
df_copy['BankLoss'].describe()

count    135109.000
mean      10493.938
std       19249.789
min           0.000
25%           0.000
50%        3484.054
75%       14437.295
max     1213184.643
Name: BankLoss, dtype: float64

In [16]:
df_copy['BankLoss%'].describe()

count   135109.000
mean        12.232
std         13.942
min          0.000
25%          0.000
50%          7.223
75%         21.458
max         69.798
Name: BankLoss%, dtype: float64

In [17]:
df_copy['Default'].describe()

count   530453.000
mean         0.257
std          0.437
min          0.000
25%          0.000
50%          0.000
75%          1.000
max          1.000
Name: Default, dtype: float64

In [18]:
df_copy['Default'].value_counts()

0.000    394129
1.000    136324
Name: Default, dtype: int64

In [19]:
df_copy['ChgOffPrinGr'].sum() / 10

1029063996.9125255

In [20]:
df_copy['ChgOffPrinGr'].sum() / df_copy['DisbursementGross'].sum()

0.08201160749979357

In [21]:
# obj_visualization(df_copy, obj_features=['State', 'BankState'])

In [22]:
print(df_copy['Is_Cross_State'].mean())
df_copy['Is_Cross_State'].value_counts()

0.5692926611782759


1    301983
0    228470
Name: Is_Cross_State, dtype: int64

In [23]:
df_nc_cross = df_copy[(df_copy['Is_Cross_State'] == 1) & (df_copy['BankState'] == 'NC')]
df_nc_cross['Bank'].value_counts()[:10]

BANK OF AMERICA NATL ASSOC        56246
BRANCH BK. & TR CO                 2175
FIRST-CITIZENS BK & TR CO          1083
WELLS FARGO BANK NATL ASSOC         412
SURREY BANK & TRUST                  50
LIVE OAK BANKING COMPANY             46
PNC BANK, NATIONAL ASSOCIATION        7
COMMUNITYONE BANK NATL ASSOC          5
CARTER BANK & TRUST                   3
UWHARRIE BANK                         3
Name: Bank, dtype: int64

In [24]:
bank_list = ['CAPITAL ONE NATL ASSOC', 'HSBC BK USA NATL ASSOC', 'BRANCH BK. & TR CO', 'FIRST-CITIZENS BK & TR CO', 
             'BANK OF AMERICA NATL ASSOC', 'WELLS FARGO BANK NATL ASSOC', 'CITIBANK, N.A.']

In [25]:
for bk in bank_list:
    print(('=' * 6) + bk + ('=' * 6))
    print('======ALL======')
    df_cpt1 = df_copy[(df_copy['Bank'] == bk)]
    display(df_cpt1['Default'].value_counts())
    print('row:', len(df_cpt1))
    print('Default rate: {:.2f}%'.format(df_cpt1['Default'].mean() * 100))
    print('======Cross State======')
    df_cpt1 = df_copy[(df_copy['Is_Cross_State'] == 1) & (df_copy['Bank'] == bk)]
    display(df_cpt1['Default'].value_counts())
    print('row:', len(df_cpt1))
    print('Default rate: {:.2f}%'.format(df_cpt1['Default'].mean() * 100))

======CAPITAL ONE NATL ASSOC======
======ALL======


0.000    12023
1.000     9390
Name: Default, dtype: int64

row: 21413
Default rate: 43.85%
======Cross State======


0.000    11652
1.000     9141
Name: Default, dtype: int64

row: 20793
Default rate: 43.96%
======HSBC BK USA NATL ASSOC======
======ALL======


...

row: 21668
Default rate: 28.84%
======CITIBANK, N.A.======
======ALL======


0.000    3857
1.000    1350
Name: Default, dtype: int64

row: 5207
Default rate: 25.93%
======Cross State======


0.000    2079
1.000    1153
Name: Default, dtype: int64

row: 3232
Default rate: 35.67%


In [26]:
df_copy['Bank'].value_counts()[:5]

BANK OF AMERICA NATL ASSOC        67893
JPMORGAN CHASE BANK NATL ASSOC    32342
CITIZENS BANK NATL ASSOC          31629
WELLS FARGO BANK NATL ASSOC       26214
BBCN BANK                         21681
Name: Bank, dtype: int64

In [27]:
for bk in list(df_copy['Bank'].value_counts()[:10].index):
    print('')
    print(('=' * 6) + bk + ('=' * 6))
    print('')
    print('======ALL======')
    df_cpt1 = df_copy[(df_copy['Bank'] == bk)]
    display(df_cpt1['Default'].value_counts())
    print('row:', len(df_cpt1))
    print('Default rate: {:.2f}%'.format(df_cpt1['Default'].mean() * 100))
    print('')
    print('======Cross State======')
    print('')
    df_cpt1 = df_copy[(df_copy['Is_Cross_State'] == 1) & (df_copy['Bank'] == bk)]
    display(df_cpt1['Default'].value_counts())
    print('row:', len(df_cpt1))
    print('Default rate: {:.2f}%'.format(df_cpt1['Default'].mean() * 100))


======BANK OF AMERICA NATL ASSOC======

======ALL======


0.000    44645
1.000    23248
Name: Default, dtype: int64

row: 67893
Default rate: 34.24%

======Cross State======



0.000    42677
1.000    22628
Name: Default, dtype: int64

row: 65305
Default rate: 34.65%

======JPMORGAN CHASE BANK NATL ASSOC======

======ALL======


...

row: 6078
Default rate: 32.13%

======MANUFACTURERS & TRADERS TR CO======

======ALL======


0.000    5310
1.000    1144
Name: Default, dtype: int64

row: 6454
Default rate: 17.73%

======Cross State======



0.000    2226
1.000     551
Name: Default, dtype: int64

row: 2777
Default rate: 19.84%


In [28]:
# df_va_cross.to_csv('VA_CrossState.csv', index=False)
# df_nc_cross.to_csv('NC_CrossState.csv', index=False)

In [29]:
# df_vis = df_copy.copy()

# x_axis = "Term"
# y_axis = "DisbursementGross"
# hue_col = 'Default'
# exclude_outlier=1

# df_vis = df_vis[~(pd.isnull(df_vis[x_axis]))]
# df_vis = df_vis[~(pd.isnull(df_vis[y_axis]))]

# upb1 = np.percentile(df_vis[x_axis], 100-exclude_outlier)
# df_vis = df_vis[(df_vis[x_axis] <= upb1)]
# upb2 = np.percentile(df_vis[y_axis], 100-exclude_outlier)
# df_vis = df_vis[(df_vis[y_axis] <= upb2)]

# df_samp = df_vis.sample(n=2000, random_state=0)

# plt.figure(figsize=(15, 12))
# sns.set_theme(style="darkgrid")
# title = 'scatter plot_{}_{}_default hue'.format(x_axis, y_axis)
# plt.title(title)
# sns.scatterplot(data=df_samp, x=x_axis, y=y_axis, hue=hue_col)
# plt.savefig(title + '.jpeg')
# plt.show()

In [30]:
# scatter_plot_with_hue(df=df_copy, x_axis = "Term", y_axis="DisbursementGross", hue_col='RevLineCr', exclude_outlier=1)

In [31]:
# scatter_plot_with_hue(df=df_copy, x_axis = "Term", y_axis="DisbursementGross", hue_col='LowDoc', exclude_outlier=1)

In [32]:
# scatter_plot_with_hue(df=df_copy, x_axis = "Term", y_axis="DisbursementGross", hue_col='RealEstate', exclude_outlier=1)

In [33]:
# scatter_plot_with_hue(df=df_copy, x_axis = "Term", y_axis="DisbursementGross", hue_col='Is_Cross_State', exclude_outlier=1)

In [34]:
# num = 'Term'
# df_vis = df_copy.copy()
# df_vis = df_vis[df_vis['ApprovalDate_year'] >= 1998]

# for i in range(len(obj_features)):
#     if obj_features[i] == 'Bank':
#         continue
#     if df_vis[obj_features[i]].nunique() < 8:
#         plt.figure(figsize=(10, 10))
#     elif obj_features[i] == 'Industry':
#         plt.figure(figsize=(20, 15))
#         vis = df_vis.groupby(by=obj_features[i]).mean().sort_values(by=num, ascending=False)
#         title = 'Mean {} by {}'.format(num, obj_features[i])
#         plt.title(title)
#         sns.set_theme(style="darkgrid")
#         plt.grid(True)
#         sns.barplot(y=vis.index, x=vis[num])
#         plt.savefig(title + '.jpeg')
#         plt.show()
#         continue
#     else:
#         plt.figure(figsize=(20, 10))
        
#     vis = df_vis.groupby(by=obj_features[i]).mean().sort_values(by=num, ascending=False)
    
#     if obj_features[i] in time_cols:
#         vis.index = [int(i) for i in vis.index]
    
#     title = 'Mean {} by {}'.format(num, obj_features[i])
#     plt.title(title)
#     sns.set_theme(style="darkgrid")
#     plt.grid(True)
#     sns.barplot(x=vis.index, y=vis[num])
#     plt.savefig(title + '.jpeg')
#     plt.show()

In [35]:
print(df_copy['BankLoss'].sum())

1417825433.05309


In [36]:
print(df_copy['ChgOffPrinGr'].sum() - df_copy['BankLoss'].sum())

8872814536.072166


In [37]:
df_bank_mean = df_groupby_mean['Bank'].loc[list(df_groupby_size['Bank'].sort_values(by='size', ascending=False)[:100].index), :]
df_bank_sum = df_groupby_sum['Bank'].loc[list(df_groupby_size['Bank'].sort_values(by='size', ascending=False)[:100].index), :]

In [38]:
rank = df_groupby_size['Bank'].sort_values(by='size', ascending=False)[:100].reset_index().reset_index()
rank['index'] = rank['index'] + 1
rank.head(3)

,index,Bank,size
0,1,BANK OF AMERICA NATL ASSOC,67893
1,2,JPMORGAN CHASE BANK NATL ASSOC,32342
2,3,CITIZENS BANK NATL ASSOC,31629


In [39]:
imp_cols = ['ChgOffPrinGr', 'Default', 'BankLoss', 'BankLoss%', 'Default%', 'SBALoss']
df_bank_mean['SBALoss'] = df_bank_mean['ChgOffPrinGr'] - df_bank_mean['BankLoss']
df_bank_sum['SBALoss'] = df_bank_sum['ChgOffPrinGr'] - df_bank_sum['BankLoss']

In [40]:
display(df_bank_mean[['ChgOffPrinGr', 'Default', 'BankLoss', 'BankLoss%', 'Default%', 'SBALoss']].head(3))
display(df_bank_sum[['ChgOffPrinGr', 'Default', 'BankLoss', 'BankLoss%', 'Default%', 'SBALoss']].head(3))

,ChgOffPrinGr,Default,BankLoss,BankLoss%,Default%,SBALoss
Bank,,,,,,
BANK OF AMERICA NATL ASSOC,37523.654,0.342,13663.646,22.156,0.589,23860.008
JPMORGAN CHASE BANK NATL ASSOC,47807.867,0.366,14232.731,15.606,0.454,33575.136
CITIZENS BANK NATL ASSOC,41141.740,0.271,13924.715,18.461,0.552,27217.025


,ChgOffPrinGr,Default,BankLoss,BankLoss%,Default%,SBALoss
Bank,,,,,,
BANK OF AMERICA NATL ASSOC,870098486.426,23248.000,316832626.932,513748.724,13650.724,553265859.494
JPMORGAN CHASE BANK NATL ASSOC,565997338.906,11849.000,168501304.299,184761.601,5377.449,397496034.607
CITIZENS BANK NATL ASSOC,313047496.616,8556.000,105953154.567,140470.807,4200.729,207094342.049


In [41]:
# for col in imp_cols:
#     print('===== {}_mean ====='.format(col))
#     temp_mean = df_bank_mean.sort_values(by=col, ascending=False).reset_index()
#     result = temp_mean.set_index('Bank').join(rank.set_index('Bank'))
#     result = result.reset_index()
#     display(result[['Bank', col, 'index', 'size']].head(5))
    
#     print('===== {}_sum ====='.format(col))
#     temp_sum = df_bank_sum.sort_values(by=col, ascending=False).reset_index()
#     result = temp_sum.set_index('Bank').join(rank.set_index('Bank'))
#     result = result.reset_index()
#     display(result[['Bank', col, 'index', 'size']].head(5))

In [42]:
# year_col_visualization(df_copy, num_features=['CreateJob', 'RetainedJob', 'USDPerJobCreated', 'Job'], year_col='DisbursementDate_year')

In [43]:
# num = 'Job'
# for i in range(len(obj_features)):
#     if obj_features[i] == 'Bank':
#         continue
#     if df_copy[obj_features[i]].nunique() < 8:
#             plt.figure(figsize=(10, 10))
#     elif obj_features[i] == 'Industry':
#         plt.figure(figsize=(20, 15))
#         vis = df_groupby_mean[obj_features[i]].sort_values(by=num, ascending=False)
#         title = 'Mean {} by {}'.format(num, obj_features[i])
#         plt.title(title)
#         sns.set_theme(style="darkgrid")
#         plt.grid(True)
#         sns.barplot(y=vis.index, x=vis[num])
#         plt.savefig(title + '.jpeg')
#         plt.show()
#         continue
#     else:
#         plt.figure(figsize=(20, 10))
#     vis = df_groupby_mean[obj_features[i]].sort_values(by=num, ascending=False)
#     title = 'Mean {} by {}'.format(num, obj_features[i])
#     plt.title(title)
#     sns.set_theme(style="darkgrid")
#     plt.grid(True)
#     sns.barplot(x=vis.index, y=vis[num])
#     plt.savefig(title + '.jpeg')
#     plt.show()

In [44]:
# plt.figure(figsize=(20, 10))
# sns.lineplot(data=pd.DataFrame(df_groupby_sum['DisbursementDate_year']['Job']).reset_index(), x='DisbursementDate_year', y='Job')
# plt.xticks(pd.DataFrame(df_groupby_sum['DisbursementDate_year']['Job']).reset_index()['DisbursementDate_year'])
# title = '2000-2009 SBA Create and Retained Job Sum'
# plt.title(title)
# plt.savefig('{}.jpeg'.format(title))
# plt.show()

In [45]:
pd.DataFrame(df_groupby_sum['DisbursementDate_year'][['CreateJob', 'RetainedJob', 'Job', 'SBALoss']])

,CreateJob,RetainedJob,Job,SBALoss
DisbursementDate_year,,,,
2000.000,109822.000,215998.000,325820.000,707161540.521
2001.000,94347.000,219341.000,313688.000,637306148.198
2002.000,107046.000,255179.000,362225.000,578801632.265
2003.000,122750.000,320475.000,443225.000,574431520.817
2004.000,131791.000,334978.000,466769.000,782748301.357
2005.000,149276.000,406617.000,555893.000,1166419170.583
2006.000,171742.000,438882.000,610624.000,1643508741.501
2007.000,180210.000,382737.000,562947.000,1719281929.606
2008.000,116118.000,222538.000,338656.000,837832490.939


In [46]:
pd.DataFrame(df_groupby_sum['DisbursementDate_year']['SBALoss'])

,SBALoss
DisbursementDate_year,
2000.000,707161540.521
2001.000,637306148.198
2002.000,578801632.265
2003.000,574431520.817
2004.000,782748301.357
2005.000,1166419170.583
2006.000,1643508741.501
2007.000,1719281929.606
2008.000,837832490.939


In [47]:
# scatter_plot_with_hue(df=df_copy, x_axis="Term", y_axis="Job", hue_col='Default', exclude_outlier=1)

In [48]:
# scatter_plot_with_hue(df=df_copy, x_axis="DisbursementGross", y_axis="Job", hue_col='Default', exclude_outlier=1)

In [49]:
# scatter_plot_with_hue(df=df_copy, x_axis="NoEmp", y_axis="Job", hue_col='NewExist', exclude_outlier=1)

In [50]:
df_bkstate_mean = df_copy.groupby(by='BankState').mean().sort_values(by=['Default', 'Is_Cross_State'], ascending=False)[['Default', 'Is_Cross_State']]
display(df_bkstate_mean[['Default', 'Is_Cross_State']].head(5))
# df_bkstate_mean.to_excel('BankState_mean.xlsx')

,Default,Is_Cross_State
BankState,,
VA,0.439,0.896
NC,0.353,0.923
SD,0.330,0.941
CA,0.321,0.458
IL,0.309,0.771


In [51]:
df_state_mean = df_copy.groupby(by='State').mean().sort_values(by=['Default', 'Is_Cross_State'], ascending=False)[['Default', 'Is_Cross_State']]
display(df_state_mean[['Default', 'Is_Cross_State']].head(5))
# df_state_mean.to_excel('State_mean.xlsx')

,Default,Is_Cross_State
State,,
GA,0.362,0.706
FL,0.362,0.812
DC,0.346,0.919
NV,0.341,0.742
TN,0.330,0.713


In [52]:
df_state_year_mean = df_copy.groupby(by=['State', 'DisbursementDate_year']).mean().reset_index()[['State', 'DisbursementDate_year', 'Default', 'Is_Cross_State']]
# df_state_year_mean.to_excel('State_year_mean.xlsx', index=False)
# scatter_plot_with_hue(df_state_year_mean, x_axis='Is_Cross_State', y_axis='Default', hue_col=None, exclude_outlier=0, sample_num=0)
df_state_year_mean.head(3)

,State,DisbursementDate_year,Default,Is_Cross_State
0,AK,2000.000,0.083,0.479
1,AK,2001.000,0.053,0.526
2,AK,2002.000,0.042,0.333


In [53]:
%run function.ipynb

In [54]:
# scatter_plot_with_hue(df_bkstate_mean, x_axis='Is_Cross_State', y_axis='Default', hue_col=None, exclude_outlier=0, sample_num=0)

In [55]:
# df_bkstate_year_mean = df_copy.groupby(by=['BankState', 'DisbursementDate_year']).mean().reset_index()[['BankState', 'DisbursementDate_year', 'Default', 'Is_Cross_State']]
# # df_bkstate_year_mean.to_excel('BankState_year_mean.xlsx', index=False)
# scatter_plot_with_hue(df_bkstate_year_mean, x_axis='Is_Cross_State', y_axis='Default', hue_col=None, exclude_outlier=0, sample_num=0)
# df_bkstate_year_mean.head(3)

In [56]:
vanc = df_copy[(df_copy['BankState'] == 'VA') | (df_copy['BankState'] == 'NC')]

In [57]:
df_year_mean = df_copy.groupby(by='DisbursementDate_year').mean().sort_values(by=['Default', 'Is_Cross_State'], ascending=False)[['Default', 'Is_Cross_State']]
# df_year_mean.to_excel('year_mean.xlsx')
df_year_mean[['Default', 'Is_Cross_State']].head(5)

,Default,Is_Cross_State
DisbursementDate_year,,
2007.000,0.449,0.677
2006.000,0.372,0.677
2008.000,0.353,0.563
2005.000,0.278,0.629
2004.000,0.201,0.582


In [58]:
bankcross = pd.DataFrame(df_copy.groupby(by=['Bank', 'Is_Cross_State']).mean().loc[list(df_groupby_size['Bank'].sort_values(by='size', ascending=False)[:100].index), :]['Default']).reset_index()
pivot_bankcross = bankcross.pivot_table(index=['Bank'], columns=['Is_Cross_State'], values='Default')
pivot_bankcross.head(3)

Is_Cross_State,0,1
Bank,,
1ST SOURCE BANK,0.140,0.174
AMERICANWEST BANK,0.091,0.208
ARVEST BANK,0.147,0.170


In [59]:
bankstatecross = pd.DataFrame(df_copy.groupby(by=['BankState', 'Is_Cross_State']).mean()['Default']).reset_index()
pivot_bankstatecross = bankstatecross.pivot_table(index=['BankState'], columns=['Is_Cross_State'], values='Default')
pivot_bankstatecross.head(3)

Is_Cross_State,0,1
BankState,,
AK,0.123,0.500
AL,0.140,0.232
AR,0.211,0.131


In [60]:
# pivot_bankstatecross.to_excel('bankstate_cross.xlsx')

In [61]:
bigbank100 = list(df_groupby_size['Bank'].sort_values(by='size', ascending=False)[:100].index)
vigncbk = list(vanc['Bank'].unique())
print(len(vigncbk))

mutualbk = []
for bk in bigbank100:
    if bk in vigncbk:
        mutualbk.append(bk)

print(len(mutualbk))

152
15


In [62]:
vanc_index = []
others_index = []

for index, row in df_copy.iterrows():
    if index % 200 == 0:
        print(index)
    if row['Bank'] in mutualbk:
        if row['BankState'] in ['VA', 'NC']:
            vanc_index.append(index)
        else:
            others_index.append(index)

400
600
800
1000
1400
1600
2000
2200
2400
2600
2800
3000
3200
3400
3800
4000
4200
4400
4600
5000
5400
5600
5800
6000
6200
6800
7200
7400
7600
7800
8200
8600
9000
9400
9600
9800
10000
10200
10400
10800
11200
11800
12000
12400
13000
13200
13800
14000
15000
15200
15400
15800
16200
16600
16800
17000
17200
17400
17600
18200
18800
19000
19200
19400
19800
20000
20400
20600
21200
21600
22000
22400
22800
23000
23200
23400
23600
23800
24000
24400
24600
24800
25400
25600
25800
26000
26400
26600
27000
27200
27600
27800
28000
28200
28400
28600
28800
29000
29400
29600
29800
30000
30200
30400
30800
31000
31200
31400
31800
32200
32600
32800
33000
33400
33800
34000
34200
34600
34800
35800
36000
36200
36400
36600
36800
37000
37200
37600
37800
38000
38200
38400
38600
38800
39000
39200
39400
40000
40200
40600
40800
41000
41400
41600
41800
42400
42600
43000
43200
43800
44000
44200
44400
44600
45000
45200
45800
46000
46200
46400
46600
46800
47200
47400
48200
48400
48800
49400
50000
50600
51200
51600
51800
5

372400
372600
372800
373000
373200
373400
373600
373800
374000
374200
374400
374600
374800
375400
375600
375800
376000
376200
376400
376600
376800
377000
377200
377400
377600
377800
378200
378400
378600
378800
379200
379400
379600
379800
380000
380200
380600
380800
381000
381200
381400
381600
381800
382000
382200
382400
382600
382800
383000
383400
383600
383800
384000
384400
384600
384800
385000
385200
385400
385600
385800
386000
386200
386400
386600
386800
387000
387200
387400
387600
387800
388000
388200
388400
388800
389000
389200
389400
389600
389800
390000
390200
390400
390600
391000
391200
391400
391600
392000
392200
392400
392600
392800
393000
393200
393400
393800
394000
394200
394600
394800
395200
395400
395600
396000
396200
396600
396800
397000
397200
397400
397600
397800
398200
398400
398600
398800
399000
399400
399600
399800
400000
400400
400600
401000
401400
401800
402000
402200
402400
402600
403000
403200
403400
403600
403800
404000
404200
404600
404800
405000
405400
405600

731600
732200
732400
732600
733600
733800
734800
735000
735200
736600
738600
739600
739800
740200
740400
740800
741200
742600
743000
743200
744600
746000
746200
746600
748000
748400
749600
749800
750200
751800
752000
752200
754000
755200
755400
755600
755800
756200
757400
757600
757800
758000
759000
759400
760400
760600
761600
763000
763200
763400
763800
764400
764600
764800
765200
767400
768000
768800
769000
769200
770000
770600
771200
771800
772800
773200
773400
773800
774000
774800
775000
775600
775800
776200
776600
776800
777000
777400
777600
777800
778000
778200
778400
778800
779000
779200
779400
779800
780200
780400
780800
781000
781600
781800
782000
782400
782800
783800
784400
784800
785000
785400
786000
786200
786400
786600
786800
787000
787600
787800
788400
789000
789200
789600
790000
790800
791600
791800
792000
792600
792800
793000
794000
794200
794600
794800
795200
795600
795800
796000
796600
797000
797200
797400
797800
798400
798600
799400
799600
800400
800600
800800
801200

In [63]:
vanc = df_copy.loc[vanc_index, :]
others = df_copy.loc[others_index, :]

In [64]:
vanc_cross_file = pd.DataFrame(vanc.groupby(by=['Bank', 'Is_Cross_State']).mean()['Default']).reset_index().pivot_table(index=['Bank'], columns=['Is_Cross_State'], values='Default')
vanc_cross_file.columns = ['vanc_0', 'vanc_1']
display(vanc_cross_file)

others_cross_file = pd.DataFrame(others.groupby(by=['Bank', 'Is_Cross_State']).mean()['Default']).reset_index().pivot_table(index=['Bank'], columns=['Is_Cross_State'], values='Default')
others_cross_file.columns = ['others_0', 'others_1']
display(others_cross_file)

,vanc_0,vanc_1
Bank,,
BANK OF AMERICA NATL ASSOC,0.316,0.381
BRANCH BK. & TR CO,0.151,0.183
"BUSINESS FINANCE GROUP, INC.",0.000,0.000
CAPITAL ONE NATL ASSOC,0.409,0.440
FIFTH THIRD BANK,0.048,0.000
FIRST BANK,0.180,0.000
FIRST-CITIZENS BK & TR CO,0.082,0.102
HERITAGE BANK,0.000,NaN
HSBC BK USA NATL ASSOC,NaN,0.944


,others_0,others_1
Bank,,
BANK OF AMERICA NATL ASSOC,0.088,0.133
BRANCH BK. & TR CO,0.073,0.140
CAPITAL ONE NATL ASSOC,0.359,0.214
FIFTH THIRD BANK,0.126,0.195
FIRST BANK,0.121,0.357
FIRST-CITIZENS BK & TR CO,0.237,0.500
HERITAGE BANK,0.144,0.167
HSBC BK USA NATL ASSOC,0.219,0.315
MANUFACTURERS & TRADERS TR CO,0.162,0.198


In [65]:
state_bank_cross_file = pd.concat([vanc_cross_file, others_cross_file], axis=1)
# state_bank_cross_file.to_excel('bankstate_bank_cross.xlsx')
state_bank_cross_file

,vanc_0,vanc_1,others_0,others_1
Bank,,,,
BANK OF AMERICA NATL ASSOC,0.316,0.381,0.088,0.133
BRANCH BK. & TR CO,0.151,0.183,0.073,0.140
"BUSINESS FINANCE GROUP, INC.",0.000,0.000,NaN,NaN
CAPITAL ONE NATL ASSOC,0.409,0.440,0.359,0.214
FIFTH THIRD BANK,0.048,0.000,0.126,0.195
FIRST BANK,0.180,0.000,0.121,0.357
FIRST-CITIZENS BK & TR CO,0.082,0.102,0.237,0.500
HERITAGE BANK,0.000,NaN,0.144,0.167
HSBC BK USA NATL ASSOC,NaN,0.944,0.219,0.315


In [66]:
vanc_file = pd.DataFrame(vanc.groupby(by=['Bank']).mean()['Default'])
vanc_file.columns = ['vanc']
display(vanc_file)

others_file = pd.DataFrame(others.groupby(by=['Bank']).mean()['Default'])
others_file.columns = ['others']
display(others_file)

,vanc
Bank,
BANK OF AMERICA NATL ASSOC,0.379
BRANCH BK. & TR CO,0.176
"BUSINESS FINANCE GROUP, INC.",0.000
CAPITAL ONE NATL ASSOC,0.439
FIFTH THIRD BANK,0.047
FIRST BANK,0.176
FIRST-CITIZENS BK & TR CO,0.098
HERITAGE BANK,0.000
HSBC BK USA NATL ASSOC,0.944


,others
Bank,
BANK OF AMERICA NATL ASSOC,0.129
BRANCH BK. & TR CO,0.080
CAPITAL ONE NATL ASSOC,0.340
FIFTH THIRD BANK,0.167
FIRST BANK,0.248
FIRST-CITIZENS BK & TR CO,0.334
HERITAGE BANK,0.144
HSBC BK USA NATL ASSOC,0.229
MANUFACTURERS & TRADERS TR CO,0.177


In [67]:
state_bank_file = pd.concat([vanc_file, others_file], axis=1)
# state_bank_file.to_excel('bankstate_bank.xlsx')
state_bank_file

,vanc,others
Bank,,
BANK OF AMERICA NATL ASSOC,0.379,0.129
BRANCH BK. & TR CO,0.176,0.080
"BUSINESS FINANCE GROUP, INC.",0.000,NaN
CAPITAL ONE NATL ASSOC,0.439,0.340
FIFTH THIRD BANK,0.047,0.167
FIRST BANK,0.176,0.248
FIRST-CITIZENS BK & TR CO,0.098,0.334
HERITAGE BANK,0.000,0.144
HSBC BK USA NATL ASSOC,0.944,0.229


In [68]:
va_all = df_copy[(df_copy['BankState'] == 'VA')][['Default', 'Is_Cross_State']]
nc_all = df_copy[(df_copy['BankState'] == 'NC')][['Default', 'Is_Cross_State']]
others_all = df_copy[(df_copy['BankState'] != 'VA') & (df_copy['BankState'] != 'NC')][['Default', 'Is_Cross_State']]
print(va_all.shape)
print(nc_all.shape)
print(others_all.shape)

(25408, 2)
(65019, 2)
(440026, 2)


In [69]:
# va_all.to_excel('va_all.xlsx', index=False)
# nc_all.to_excel('nc_all.xlsx', index=False)
# others_all.to_excel('others_all.xlsx', index=False)

In [116]:
df_copy

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,RealEstate,GreatRecession,DotCom,Recession,Mon_Is_14710,DisbursePerMonth,BankLoss,BankLoss%,Job,SBALoss
12,1000146010,CARVEL,APEX,NC,27502,STEARNS BK NATL ASSOC,MN,2006-02-07,162.000,2.000,...,0.000,0.000,0.000,0.000,0.000,1896.146,NaN,NaN,0.000,NaN
24,1000446006,SUBWAY,LITTLE ROCK,AR,72223,HOPE FCU,MS,2006-02-07,126.000,7.000,...,0.000,0.000,0.000,0.000,1.000,1320.931,NaN,NaN,0.000,NaN
28,1000506000,WEYLAND CORPORATION,CAMARILLO,CA,93010,WELLS FARGO BANK NATL ASSOC,SD,2006-02-07,83.000,18.000,...,0.000,0.000,0.000,0.000,0.000,6404.900,NaN,NaN,28.000,NaN
30,1000516003,CHICAGO BRICK UNLIMITED INC,MIAMI,FL,33186,"CITIBANK, N.A.",FL,2006-02-07,84.000,4.000,...,0.000,0.000,0.000,0.000,0.000,742.338,NaN,NaN,4.000,NaN
32,1000526006,"RZI, INC.",NEW ORLEANS,LA,70130,BUSINESS RES. CAP. SPECIALTY B,LA,2006-02-07,60.000,3.000,...,0.000,0.000,0.000,0.000,0.000,1010.181,NaN,NaN,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855582,9991423008,"THE AUTO CLINIC, INC.",LEES SUMMIT,MO,64081,"RURAL MISSOURI, INC.",MO,1997-02-26,240.000,9.000,...,1.000,0.000,0.000,0.000,0.000,1723.132,NaN,NaN,9.000,NaN
855584,9991463009,"A.S.A.P. CLEANING SERVICES, IN",STEELTON,PA,17113,"PNC BANK, NATIONAL ASSOCIATION",PA,1997-02-26,12.000,17.000,...,0.000,0.000,1.000,1.000,0.000,1257.761,NaN,NaN,0.000,NaN
855624,9992673004,UNCLE O'S,BATON ROUGE,LA,70817,JPMORGAN CHASE BANK NATL ASSOC,LA,1997-02-27,84.000,14.000,...,0.000,0.000,0.000,0.000,0.000,1796.801,NaN,NaN,0.000,NaN
855706,9994003006,"FIVE STAR ELECTRIC, INC.",PHILADELPHIA,PA,19145,WELLS FARGO BANK NATL ASSOC,PA,1997-02-27,36.000,5.000,...,0.000,0.000,1.000,1.000,1.000,1257.761,NaN,NaN,0.000,NaN


In [117]:
rate = pd.read_excel('fed-funds-rate-historical-chart.xlsx')
rate = rate.dropna()
rate.head(5)

,date,value
0,1971-01-01,3.000
1,1971-01-02,3.000
2,1971-01-03,3.000
3,1971-01-04,5.500
4,1971-01-05,5.500


In [126]:
model_demo = df_copy.drop(['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'CreateJob', 'RetainedJob', 'ChgOffDate', 'BalanceGross',
                           'ChgOffDate_year', 'ChgOffDate_month', 'ChgOffDate_day',  'HousingUnits', 
                           'OccupiedHousingUnits', 'MedianHomeValue', 'MedianHouseholdIncome', 'USDPerJobCreated', 
                           'loan_marturity_date', 'Default%', 'BankLoss', 'BankLoss%', 'Job', 'SBALoss', 'Bank'], axis=1)

In [127]:
model_demo['ApprovalDate'] = model_demo['ApprovalDate'].astype('datetime64')

In [128]:
model_demo

,State,BankState,ApprovalDate,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,...,Bank%,State_division,Is_Cross_State,Appv_Dusburse_delta,RealEstate,GreatRecession,DotCom,Recession,Mon_Is_14710,DisbursePerMonth
12,NC,MN,2006-02-07,162.000,2.000,0.000,1.000,1,0.000,0.000,...,25.000,South East,1,52.000,0.000,0.000,0.000,0.000,0.000,1896.146
24,AR,MS,2006-02-07,126.000,7.000,1.000,0.000,1,0.000,0.000,...,15.000,South East,1,82.000,0.000,0.000,0.000,0.000,1.000,1320.931
28,CA,SD,2006-02-07,83.000,18.000,0.000,0.000,1,1.000,0.000,...,50.000,West,1,21.000,0.000,0.000,0.000,0.000,0.000,6404.900
30,FL,FL,2006-02-07,84.000,4.000,1.000,0.000,1,1.000,0.000,...,50.000,South East,0,21.000,0.000,0.000,0.000,0.000,0.000,742.338
32,LA,LA,2006-02-07,60.000,3.000,1.000,0.000,1,0.000,0.000,...,15.000,South East,0,113.000,0.000,0.000,0.000,0.000,0.000,1010.181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855582,MO,MO,1997-02-26,240.000,9.000,1.000,0.000,-1,0.000,0.000,...,0.000,Mid-West,0,1085.000,1.000,0.000,0.000,0.000,0.000,1723.132
855584,PA,PA,1997-02-26,12.000,17.000,1.000,0.000,-1,1.000,0.000,...,50.000,Mid-Atlantic,0,1129.000,0.000,0.000,1.000,1.000,0.000,1257.761
855624,LA,LA,1997-02-27,84.000,14.000,0.000,0.000,-1,0.000,1.000,...,20.000,South East,0,1281.000,0.000,0.000,0.000,0.000,0.000,1796.801
855706,PA,PA,1997-02-27,36.000,5.000,1.000,0.000,-1,1.000,0.000,...,50.000,Mid-Atlantic,0,1523.000,0.000,0.000,1.000,1.000,1.000,1257.761


In [129]:
model_data = pd.merge(model_demo, rate, left_on='ApprovalDate', right_on='date', how='left')
model_data

,State,BankState,ApprovalDate,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,...,Is_Cross_State,Appv_Dusburse_delta,RealEstate,GreatRecession,DotCom,Recession,Mon_Is_14710,DisbursePerMonth,date,value
0,NC,MN,2006-02-07,162.000,2.000,0.000,1.000,1,0.000,0.000,...,1,52.000,0.000,0.000,0.000,0.000,0.000,1896.146,2006-02-07,4.470
1,AR,MS,2006-02-07,126.000,7.000,1.000,0.000,1,0.000,0.000,...,1,82.000,0.000,0.000,0.000,0.000,1.000,1320.931,2006-02-07,4.470
2,CA,SD,2006-02-07,83.000,18.000,0.000,0.000,1,1.000,0.000,...,1,21.000,0.000,0.000,0.000,0.000,0.000,6404.900,2006-02-07,4.470
3,FL,FL,2006-02-07,84.000,4.000,1.000,0.000,1,1.000,0.000,...,0,21.000,0.000,0.000,0.000,0.000,0.000,742.338,2006-02-07,4.470
4,LA,LA,2006-02-07,60.000,3.000,1.000,0.000,1,0.000,0.000,...,0,113.000,0.000,0.000,0.000,0.000,0.000,1010.181,2006-02-07,4.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530448,MO,MO,1997-02-26,240.000,9.000,1.000,0.000,-1,0.000,0.000,...,0,1085.000,1.000,0.000,0.000,0.000,0.000,1723.132,1997-02-26,5.200
530449,PA,PA,1997-02-26,12.000,17.000,1.000,0.000,-1,1.000,0.000,...,0,1129.000,0.000,0.000,1.000,1.000,0.000,1257.761,1997-02-26,5.200
530450,LA,LA,1997-02-27,84.000,14.000,0.000,0.000,-1,0.000,1.000,...,0,1281.000,0.000,0.000,0.000,0.000,0.000,1796.801,1997-02-27,5.290
530451,PA,PA,1997-02-27,36.000,5.000,1.000,0.000,-1,1.000,0.000,...,0,1523.000,0.000,0.000,1.000,1.000,1.000,1257.761,1997-02-27,5.290


In [131]:
model_data = model_data.drop(['ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'DisbursementDate', 'ApprovalDate', 'DisbursementDate_year', 
                              'DisbursementDate_month','DisbursementDate_day', 'ApprovalDate_year', 'ApprovalDate_month', 
                              'ApprovalDate_day', 'Population', 'PopDensity', 'GreatRecession', 'DotCom', 'date', 'BankState'], axis=1)

In [136]:
model_data.rename(columns={' value': 'rate'}, inplace=True)

In [137]:
model_data.columns

Index(['State', 'Term', 'NoEmp', 'NewExist', 'IsFranchised', 'UrbanRural',
       'RevLineCr', 'LowDoc', 'DisbursementGross', 'Default', 'Industry',
       'DisbursePerEmployee', 'Bank%', 'State_division', 'Is_Cross_State',
       'Appv_Dusburse_delta', 'RealEstate', 'Recession', 'Mon_Is_14710',
       'DisbursePerMonth', 'rate'],
      dtype='object')

In [145]:
model_data = model_data[model_data['UrbanRural'] != -1]

In [146]:
model_data

,State,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,DisbursementGross,Default,...,DisbursePerEmployee,Bank%,State_division,Is_Cross_State,Appv_Dusburse_delta,RealEstate,Recession,Mon_Is_14710,DisbursePerMonth,rate
0,NC,162.000,2.000,0.000,1.000,1,0.000,0.000,307175.719,0.000,...,84466.667,25.000,South East,1,52.000,0.000,0.000,0.000,1896.146,4.470
1,AR,126.000,7.000,1.000,0.000,1,0.000,0.000,166437.357,0.000,...,17162.500,15.000,South East,1,82.000,0.000,0.000,1.000,1320.931,4.470
2,CA,83.000,18.000,0.000,0.000,1,1.000,0.000,531606.737,0.000,...,23081.105,50.000,West,1,21.000,0.000,0.000,0.000,6404.900,4.470
3,FL,84.000,4.000,1.000,0.000,1,1.000,0.000,62356.429,0.000,...,10288.000,50.000,South East,0,21.000,0.000,0.000,0.000,742.338,4.470
4,LA,60.000,3.000,1.000,0.000,1,0.000,0.000,60610.836,0.000,...,12500.000,15.000,South East,0,113.000,0.000,0.000,0.000,1010.181,4.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530371,CO,84.000,2.000,1.000,0.000,0,1.000,0.000,36366.502,0.000,...,10000.000,50.000,South West,1,22.000,0.000,0.000,0.000,432.935,4.510
530372,AZ,29.000,2.000,0.000,0.000,1,-1.000,0.000,206076.844,1.000,...,56666.667,25.000,South West,1,22.000,0.000,0.000,0.000,7106.098,4.510
530373,NV,240.000,20.000,1.000,0.000,1,-1.000,0.000,617018.315,0.000,...,24238.095,0.000,West,0,464.000,1.000,0.000,0.000,2570.910,4.510
530374,AZ,84.000,3.000,1.000,0.000,1,1.000,0.000,18953.009,0.000,...,3908.750,50.000,South West,1,22.000,0.000,0.000,0.000,225.631,4.510


In [147]:
model_data.to_csv('model1.csv', index=False)